## 02.06.2023 ROC, k-nearest neighbour, decision trees and random forests

Copyright (C) 2023, B. Zeller-Plumhoff

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the [GNU General Public License](https://www.gnu.org/licenses/gpl-3.0.html) for more details.

This Jupyter Notebook was created by Berit Zeller-Plumhoff for the course "Data Science for Material Scientists" at Kiel University. 

Within the notebook, you will perform a classification of metallicity using logistic regression, $k$-nearest neighbours and decision trees. Varying the $k$ and depth of the tree, you will assess how these influence your results and compare the performance of the different classifiers using ROC metrics and curves.

<span style='color:red'> 02.06.2023: You will also include random forests classifiers in this notebook this week. </span>

We begin by loading the required libraries.

In [ ]:
import pandas as pd # library for organizing data
import numpy as np # library for numerial computations
from sklearn import linear_model # the linear_model library establishes a straightforward implementation of a linear regression model
from sklearn.model_selection import train_test_split, KFold, cross_val_score # this library enables the splitting of a data set into training and test data
from sklearn.inspection import DecisionBoundaryDisplay # library to display decision boundaries of classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, RocCurveDisplay, roc_curve, roc_auc_score, log_loss, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt # library for plotting (not interactive)
import matminer.datasets as mm # library for data mining materials properties, accesses published datasets
from matminer.featurizers.conversions import StrToComposition # converts a string denoting a material composition into the composition
from matminer.featurizers.composition.element import ElementFraction # determines the element fraction for a given composition
from pymatgen.core import Composition # materials analysis library, module used to analyse the chemical composition of a compound

### Predicting metallicity

For classification of metallicit based on the material compositon, we will use the dataset based on the publication from [Zhuo et al., 2018](https://pubs.acs.org/doi/pdf/10.1021/acs.jpclett.8b00124)<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) that can be accessed through __matminer__. Have a look at last week's notebook to learn more about the dataset and __matminer__.

In this case, we will start by loading the pickle file which we prepared previously.

In [ ]:
df=pd.read_pickle('.\ismetal_df.pkl')
df.head()

Assign the dataframe __without__ columns "formula" "is_metal" and "composition" to your feature variable X and the "is_metal" column to the observation y. Then split this data into training and test data.

In [ ]:
# assign the variables
X=df.drop(["formula","is_metal","composition"],axis=1)
y=df["is_metal"]

# split your data sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

Now we will sort the data we created above using different algorithms. To do so, use the train data to fit and the test data to predict the classifiers. Do this by employing the [__logistic regression__](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression), the [__k-nearest neighbors__](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and the [__decision tree__](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) classification methods. In the case of classification using __k-nearest neighbors__, we will make predictions considering different numbers of nearest neighbors. The same will be done for the __decision tree__ and the maximum depth of the tree. This can be performed within a loop, where you will __append__ the current model and predictions to a list thereof, which will initially be empty. For the loop, consider the range from $\left[1, 20\right]$. Read the documentation for these methods if necessary. 

<span style='color:red'> 02.06.2023: add the [__random forest__](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#) classifier, looping over the number of estimators to include in the ensemble from $\left[1, 100\right]$ in steps of 10. </span>

In [ ]:
# logistic regression
model_log_reg = linear_model.LogisticRegression()
model_log_reg=model_log_reg.fit(X_train, y_train)
y_pred_log_reg = model_log_reg.predict(X_test)

# k-nearest neighbors (loop over k)
model_knn=[]
y_pred_knn=[]
for k in range(1,21):
    model_knn_tmp = KNeighborsClassifier(n_neighbors=k)
    model_knn.append(model_knn_tmp.fit(X_train, y_train))
    y_pred_knn.append(model_knn_tmp.predict(X_test))

# decision tree (loop over max_depth)
model_tree=[]
y_pred_tree=[]
for i in range(1,21):
    model_tree_tmp = DecisionTreeClassifier(max_depth=i)
    model_tree.append(model_tree_tmp.fit(X_train, y_train))
    y_pred_tree.append(model_tree_tmp.predict(X_test))
    
# random forests (loop over n_estimators in steps of 10, i.e. 1, 10, 20, 30...)


### Confusion matrices

Now that we have made our predictions with the different classifiers, we are interested in checking the generalization performance of our models. This will be done using the [__confusion matrix__](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay). To do this, use the metrics module in the scikit library to create confusion matrices for our train and test data. Build confusion matrices for the three types of classifiers used above. You can also observe what happens when you consider different numbers of neighbors (for the k-nearest neighbors) and different tree depths (for the decision tree). What can you conclude from these different results?

<span style='color:red'> 02.06.2023: Add the confusion matrices for the random forests with different numbers of estimators too. How do they behave in comparison to the other classifiers? </span>

In [ ]:
# display the confusion matrices in one figure as three subplots in two rows, one for training and one for testing data
k=5
depth=10
n_est=

# display the confusion matrices in one figure as three subplots in two rows, one for training and one for testing data
fig,ax=plt.subplots(2,4, figsize=(16,8))
plt.subplots_adjust(wspace=0.4)
cm_display_log_reg = ConfusionMatrixDisplay.from_estimator(model_log_reg,X_train,y_train, normalize='all', colorbar=False,ax=ax[0][0])
ax[0][0].set_title('Logistic regression')
cm_display_knn = ConfusionMatrixDisplay.from_estimator(model_knn[k-1],X_train,y_train, normalize='all', colorbar=False,ax=ax[0][1])
ax[0][1].set_title('k-nearest neighbours')
cm_display_tree = ConfusionMatrixDisplay.from_estimator(model_tree[depth-1],X_train,y_train, normalize='all', colorbar=False,ax=ax[0][2])
ax[0][2].set_title('Decision tree')

cm_display_log_reg = ConfusionMatrixDisplay.from_estimator(model_log_reg,X_test,y_test, normalize='all', colorbar=False,ax=ax[1][0])
cm_display_knn = ConfusionMatrixDisplay.from_estimator(model_knn[k-1],X_test,y_test, normalize='all', colorbar=False,ax=ax[1][1])
cm_display_tree = ConfusionMatrixDisplay.from_estimator(model_tree[depth-1],X_test,y_test, normalize='all', colorbar=False,ax=ax[1][2])

plt.show()

#### Metrics

Based on the results obtained from the confusion matrices, we will build a panda datraframe to visualize the metrics obtained for our test data predictions. Its table should contain the following columns: 'Classifier', 'True negatives', 'False positives', 'False negatives', 'True positives', 'Positive predictive value', 'Negative predictive value', 'True positive rate', 'True negative rate', 'Log loss error', 'Accuracy', 'Area under the curve'. Do this for all four types of classifiers used. Read the documentation found in the [__sklearn.metrics.confusion_matrix__](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) module if needed and search for the other metrics accordingly.

To start, you will need to initialize a dataframe with the column names and then use the __.loc__ function to add rows.

How do the numerical values compare to the confusion matrices you had displayed?

In [ ]:
# set k and max_depth and n_estimators


# initialize the pandas dataframe with the correct column names

# compute the metrics for each classifier and add the corresponding rows to the dataframe

# display the data frame



#### ROC curves

Now we will use the ROC curves to analyze the performance of our classifiers for prediction. For this task you will need to calculate the probability estimate for all the binary classes of the model. See documentation for the methods predict_proba(...), roc_curve(...) and RocCurveDisplay(...).

1. First use the k-nearest neighbors classifier and plot the ROC curve for models where k = 1, 5, 9, 13, 17. How does the number of nearest neighbors influence the ROC curve? 

2. Do the same thing for the decision tree classifier. How does the max_depth influences the ROC curves?

3. Similarly, display the ROC curve for the random forest classifier for different n_estimators. How does this variable influence the ROC curves?

4. Finally, plot the ROC curves for the logistic regression classifier, k-nearest neighbors classifier (for k = 5), the decision tree classifier (for max_depth = 11) and for the random forest classifier (n_estimators=100).  Of course, you can play with the model parameters or divide between testing and training performance. Which differences do you notice between the different  models?

Overall, how do the ROC curves compare to the metrics you had displayed in the dataframe above?

Using the following cell, you can display the decision tree that you have trained. Run the code for different __max_depth__ and see how your tree changes. What do you conclude from the tree structure?

In [ ]:
from sklearn import tree
clf=model_tree[2]
y_names=['no metal','metal']
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf,  
                   feature_names=df.drop(["formula","is_metal","composition"],axis=1).columns, 
                   class_names=y_names,
                   filled=True)
#fig.savefig("decision_tree_depth%d.png" % i)

With trees and random forests we can determine the [__feature_importances__](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html) in a straightforward manner. The following cell lets you compute the feature importances based on the mean decrease in Gini impurity for the random forests you have trained and plot them in descending order. How does this agree with the tree you plotted in the previous cell?

In [ ]:
feature_names=df.drop(["formula","is_metal","composition"],axis=1).columns
importances = model_rf[2].feature_importances_
std = np.std([tree.feature_importances_ for tree in model_rf[2].estimators_], axis=0)
forest_importances = pd.Series(importances, index=feature_names)
sorted_idx = (-importances).argsort()[:20]

fig, ax = plt.subplots(figsize=(4,3))

ax.set_title("Feature importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

To compute the proximity matrix based on your random forests, use the following cell. The code was taken from [this stackoverflow entry](https://stackoverflow.com/questions/18703136/proximity-matrix-in-sklearn-ensemble-randomforestclassifier). You can vary which random forest model you use and have a look at the resulting visualization of proximities.

In [ ]:
def proximityMatrix(model, X, normalize=True):      

    terminals = model.apply(X)
    nTrees = terminals.shape[1]

    a = terminals[:,0]
    proxMat = 1*np.equal.outer(a, a)

    for i in range(1, nTrees):
        a = terminals[:,i]
        proxMat += 1*np.equal.outer(a, a)

    if normalize:
        proxMat = proxMat / nTrees

    return proxMat   


pM=proximityMatrix(model_rf[2], X, normalize=True)

The proximity matrix $PM$ can be converted into a dissimilarity matrix $DM=1-PM$ and using multidimensional scaling, it can then be projected onto a 2D-plane, as described in [Gilles Louppe - Understanding Random Forests: From Theory to Practice](https://arxiv.org/abs/1407.7502).
To do so, we used the sklearn method [MDS](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)
and adapted the transformation and plotting routine described [here](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py). Execute the following two cells to visualize the proximities for our classes metals and non-metals.

What does the result suggest?

In [ ]:
from sklearn import manifold
transformer=manifold.MDS(n_components=2, dissimilarity='precomputed', n_init=1, normalized_stress="auto")
transformer.dissimilarity_matrix_=(1-pM)

projections = transformer.fit_transform((1-pM))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def plot_embedding(XX, title):
    _, ax = plt.subplots(figsize=(6,6))
    XX = MinMaxScaler().fit_transform(XX)

    for ff in [True,False]:
        ax.scatter(
            *XX[y == ff].T,
            label=ff,
            s=60,
            alpha=0.425,
            zorder=2,
        )
    ax.legend()
    ax.set_title(title)
    ax.axis("off")

plot_embedding(projections,'Random forest proximities metallicity')

plt.show()

For comparison, you can run the cell below which will compute the proximity matrix and display the results for the digits data set shown in the lecture. This cell is adapted from [the scikit documentation](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py), which is available under the BSD license (© 2007 - 2023, scikit-learn developers).

In [ ]:
from sklearn.datasets import load_digits
def plot_embeddingd(Xd, title):
    _, ax = plt.subplots(figsize=(6,6))
    Xd = MinMaxScaler().fit_transform(Xd)

    for digit in digits.target_names:
        ax.scatter(
            *Xd[yd == digit].T,
            label=f"${digit}$",
            s=30,
            color=plt.cm.Set3(digit),
            zorder=2,
        )
    ax.legend(loc='lower right', bbox_to_anchor=(1.15, 0.025))
    ax.set_title(title)
    ax.axis("off")
    
digits = load_digits()
Xd, yd = digits.data, digits.target
n_samples, n_features = Xd.shape
rf_model=RandomForestClassifier(n_estimators=200)
rf_model.fit(Xd, yd)

pMd=proximityMatrix(rf_model, Xd, normalize=True)

transformerd=manifold.MDS(n_components=2, dissimilarity='precomputed', n_init=1, normalized_stress="auto")
transformerd.dissimilarity_matrix_=(1-pMd)

projectionsd = transformerd.fit_transform((1-pMd))

plot_embeddingd(projectionsd,'Random forest proximities digit data set')

plt.show()

<a name="cite_note-1"></a>1.[^](#cite_ref-1) Y. Zhuo, A.M. Tehrani, and J. Brgoch, J. Phys. Chem. Lett. 2018, 9, 7, 1668–1673, https://doi.org/10.1021/acs.jpclett.8b00124